# Text Representation
This notebook demonstrates concepts in text representation and statistical models for text analysis. It includes examples of creating a Document Term Matrix (DTM), calculating word probabilities, and applying the Multinomial Model to predict text origins.

## Preparation
### Required Packages
First, install and import the required packages for text analysis.

### Document Term Matrix
The `Document Term Matrix` is a bag-of-words representation of text. It is a matrix where:

- Rows correspond to documents.
- Columns correspond to terms (unique words in the corpus).

For example:

$$
X = \begin{bmatrix}
1 & 1 & ... \\
0 & 2 & ... \\
... & ... & ...
\end{bmatrix}
$$
represents a document-term matrix.

In [4]:
# Install necessary packages
# !pip install nltk numpy pandas scikit-learn

# Import packages
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from math import factorial


## Text Representation
### Creating a Sample Corpus

In [2]:
# Define sample texts
texts = [
    'John loves icecream',
    'John loves oranges',
    'Marry hates icecream'
]

# Check corpus size
corpus_size = len(texts)
print(f'Corpus size: {corpus_size}')

Corpus size: 3


In [3]:
def create_dtm(texts):

    if not texts or not all(isinstance(t, str) for t in texts):
        raise ValueError("Input must be a list of non-empty strings.")
    
    # Convert texts to lowercase
    texts = [text.lower() for text in texts]
    
    # Extract unique terms
    all_words = [word for text in texts for word in text.split()]
    unique_terms = sorted(set(all_words))  # Sorted for consistent column order
    
    # Create the matrix
    dtm = []
    for text in texts:
        row = [text.split().count(term) for term in unique_terms]
        dtm.append(row)
    
    return dtm, unique_terms

dtm, unique_terms = create_dtm(texts)

print("Document Term Matrix:")
for row in dtm:
    print(row)

print("\nUnique Terms (Columns):", unique_terms)


Document Term Matrix:
[0, 1, 1, 1, 0, 0]
[0, 0, 1, 1, 0, 1]
[1, 1, 0, 0, 1, 0]

Unique Terms (Columns): ['hates', 'icecream', 'john', 'loves', 'marry', 'oranges']


### Document Term Matrix

In [6]:
# Create a Document Term Matrix using CountVectorizer
vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform(texts).toarray()

# Display the matrix and terms
print('Document Term Matrix:')
print(dtm)
print('Terms:', vectorizer.get_feature_names_out())

Document Term Matrix:
[[0 1 1 1 0 0]
 [0 0 1 1 0 1]
 [1 1 0 0 1 0]]
Terms: ['hates' 'icecream' 'john' 'loves' 'marry' 'oranges']


### Task 1
Create a `Document Term Matrix` for the following texts:

In [7]:
# Define new sample texts
new_texts = [
    'banana banana banana banana chocolate',
    'chocolate chocolate chocolate banana',
    'banana banana'
]

# Create a Document Term Matrix for new texts
new_dtm = vectorizer.fit_transform(new_texts).toarray()

print('Document Term Matrix for new texts:')
print(new_dtm)
print('Terms:', vectorizer.get_feature_names_out())

Document Term Matrix for new texts:
[[4 1]
 [1 3]
 [2 0]]
Terms: ['banana' 'chocolate']


## Multinomial Model of Texts
The Multinomial Model of Texts is a probabilistic model used to determine the likelihood of a document being generated by a given source based on word probabilities. It is based on the Multinomial Distribution:

$$
P_n(x_1, x_2, ..., x_n) = \frac{N!}{x_1! x_2! ... x_n!} p_1^{x_1} \times p_2^{x_2} \times ... \times p_n^{x_n}
$$

- $N$ is the total number of words in the document.
- $x_i$ is the count of the $i$th word.
- $p_i$ is the probability of the $i$th word.


### Generating Text Data

Next, we assume that texts 1 through 3 are generated by John, while texts 4 though 6 are generated by Mary. Text_7 and Text_8 are of unknown origin. And we must determine who said what!

Create a Document Term matrix and call it banana_data:

Next, we need to calculate John and Mary word rates (how often do they use certain words).

We do this by summing together all document rates that belong to John and to Mary, respectively.

In [8]:
# Define a larger corpus
banana_texts = [
    'banana banana banana banana chocolate',
    'banana banana',
    'chocolate chocolate chocolate banana fudge',
    'icecream icecream fudge icecream',
    'fudge fudge fudge',
    'icecream icecream fudge fudge',
    'icecream fudge fudge',
    'chocolate chocolate banana banana'
]

# Create a Document Term Matrix
banana_data = vectorizer.fit_transform(banana_texts).toarray()

print('Document Term Matrix:')
print(banana_data)
print('Terms:', vectorizer.get_feature_names_out())

Document Term Matrix:
[[4 1 0 0]
 [2 0 0 0]
 [1 3 1 0]
 [0 0 1 3]
 [0 0 3 0]
 [0 0 2 2]
 [0 0 2 1]
 [2 2 0 0]]
Terms: ['banana' 'chocolate' 'fudge' 'icecream']


In [17]:
banana_data

array([[4, 1, 0, 0],
       [2, 0, 0, 0],
       [1, 3, 1, 0],
       [0, 0, 1, 3],
       [0, 0, 3, 0],
       [0, 0, 2, 2],
       [0, 0, 2, 1],
       [2, 2, 0, 0]])

In [9]:
john_docs = banana_data[0:3, :]

john_docs

array([[4, 1, 0, 0],
       [2, 0, 0, 0],
       [1, 3, 1, 0]])

In [10]:
john_rates = john_docs.sum(axis=0)

john_rates

array([7, 4, 1, 0])

we want to calculate the relative rates (probabilities) of word use. We can do this by dividing the raw rates we have by the total number of words

In [11]:
total_john = john_rates.sum()
total_john

probs_john = john_rates / total_john
probs_john

array([0.58333333, 0.33333333, 0.08333333, 0.        ])

Now, do the same for Mary

In [12]:
# your code here
mary_docs = banana_data[3:6, :]

In [13]:
mary_docs

array([[0, 0, 1, 3],
       [0, 0, 3, 0],
       [0, 0, 2, 2]])

In [17]:
mary_rates = mary_docs.sum(axis=0)
mary_rates

array([0, 0, 6, 5])

In [18]:
total_mary = mary_rates.sum()
total_mary

11

In [19]:
probs_mary = mary_rates / total_mary
probs_mary

array([0.        , 0.        , 0.54545455, 0.45454545])

## Now we have our Language Models
Next, we need to determine the rates of the unknown texts! Remember, the texts are 7 and 8.

In [28]:
unknown_1 = banana_data[6, ]
unknown_2 = banana_data[7, ]

In [30]:
print(unknown_1)

print(unknown_2)

[0 0 2 1]
[2 2 0 0]


We can now start calculating!

First, let’s determine the probability of both of these texts belonging to John.

These are John’s probabilities:

In [31]:
probs_john

array([0.58333333, 0.33333333, 0.08333333, 0.        ])

### Example: Calculating Probabilities for Unknown Texts
For an unknown text with word counts $[b=0, c=0, f=2, i=1]$, and John's probabilities:

$$
P(b=0, c=0, f=2, i=1) = \frac{3!}{0! 0! 2! 1!} \cdot 0.58^0 \cdot 0.33^0 \cdot 0.08^2 \cdot 0^1
$$
Step-by-step:

1. Calculate the factorial term:

$$
\frac{3!}{0! 0! 2! 1!}
$$
2. Multiply by the word probabilities raised to their respective counts.

In [20]:
first_part = factorial(3) / (factorial(0) * factorial(0) * factorial(2) * factorial(1))

In [37]:
first_part

3.0

Now this: 

$$
0.58^0 \cdot 0.33^0 \cdot 0.08^2 \cdot 0^1
$$

In [38]:
second_part = 0.58**0 * 0.33**0 * 0.08**2 * 0**1

Combining together:

In [40]:
first_part * second_part

0.0

This is the probability of the first `unknown` text being generated by john.

Now, let's do the second `unknown` text:

$$
P(b=2, c=2, f=0, i=0) = \frac{4!}{2!2!0!0!}0.58^{2}\times 0.33^{2} \times 0.8^0 \times 0^0
$$

In [42]:
first_part = factorial(4) / (factorial(2) * factorial(2) * factorial(0) * factorial(0))

In [43]:
# Element-wise exponentiation and product
second_part = np.prod(probs_john ** unknown_2)

In [44]:
first_part * second_part

0.22685185185185192

# Now calculate Mary probabilities yourself

### Example: Calculating Probabilities Using Multinomial Model

In [21]:
# Assuming John generated texts 1-3 and Mary generated texts 4-6
john_docs = banana_data[:3, :]
mary_docs = banana_data[3:6, :]

# Calculate word probabilities for John and Mary
john_rates = john_docs.sum(axis=0)
mary_rates = mary_docs.sum(axis=0)

john_probs = john_rates / john_rates.sum()
mary_probs = mary_rates / mary_rates.sum()

print('John Probabilities:', john_probs)
print('Mary Probabilities:', mary_probs)




# Define unknown texts (rows 7 and 8)
unknown_1 = banana_data[6, :]
unknown_2 = banana_data[7, :]

# Display unknown texts
print('Unknown Text 1:', unknown_1)
print('Unknown Text 2:', unknown_2)

# Calculate probabilities
def multinomial_probability(word_counts, word_probs):
    N = sum(word_counts)
    numerator = factorial(N)
    denominator = np.prod([factorial(c) for c in word_counts])
    probs = np.prod(word_probs ** word_counts)
    return (numerator / denominator) * probs

# Example for Unknown Text 1
p_john_unknown_1 = multinomial_probability(unknown_1, john_probs)
p_john_unknown_2 = multinomial_probability(unknown_2, john_probs)

# Similarly, calculate for Mary
p_mary_unknown_1 = multinomial_probability(unknown_1, mary_probs)
p_mary_unknown_2 = multinomial_probability(unknown_2, mary_probs)

print(f'Probability that Unknown 1 was generated by John: {p_john_unknown_1}')
print(f'Probability that Unknown 1 was generated by Mary: {p_mary_unknown_1}')

print(f'Probability that Unknown 2 was generated by John: {p_john_unknown_2}')
print(f'Probability that Unknown 2 was generated by Mary: {p_mary_unknown_2}')

John Probabilities: [0.58333333 0.33333333 0.08333333 0.        ]
Mary Probabilities: [0.         0.         0.54545455 0.45454545]
Unknown Text 1: [0 0 2 1]
Unknown Text 2: [2 2 0 0]
Probability that Unknown 1 was generated by John: 0.0
Probability that Unknown 1 was generated by Mary: 0.4057099924868519
Probability that Unknown 2 was generated by John: 0.22685185185185192
Probability that Unknown 2 was generated by Mary: 0.0


### Predicting Text Origin with the Multinomial Model

### Task: Calculate Probabilities for Mary

In [ ]:
# Use the same process to calculate probabilities for Mary.
# Define unknown texts (rows 7 and 8) and use Mary probabilities.
# Mary probabilities are already calculated as `mary_probs`. Proceed as done for John.
